In [1]:
#math
import numpy as np
from numpy import *
from scipy.optimize import curve_fit

#plotting
import matplotlib.pyplot as plt
import matplotlib

#matlab import
import scipy.io
from scipy.io import savemat

#file_store
import os

#time
from datetime import datetime, timedelta
import time

from ctypes import *
%matplotlib inline

C:\Users\Demag\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def linear_func(x, a, b):
        return a * x + b

def linear_fit(x, y):
        popt, pcov = curve_fit(linear_func, x, y)
        a = popt[0]
        b = popt[1]
        return a, b

In [3]:
initial_folder = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\CoSi\Open_loop\error_func\with_TWPA'
files = os.listdir(initial_folder)
for i in range(len(files)):
    if ".fig" in files[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', files[i]) 

[0] -----------> AnalysisScript.m
[1] -----------> AnalysisScriptWithoutTWPA.asv
[2] -----------> AnalysisScriptWithoutTWPA.m
[3] -----------> AnalysisScriptWithTWPA.asv
[4] -----------> AnalysisScriptWithTWPA.m
[5] -----------> ModulationFTest_16-43-55.mat
[6] -----------> ModulationFTest_17-08-29.mat
[7] -----------> ModulationFTest_17-13-50.mat
[8] -----------> No_Cavity_Y_ErrPsw_0to10_13-33-24.mat
[9] -----------> No_Cavity_Y_ErrPsw_0to10_16-50-38.mat
[10] -----------> No_Cavity_Y_ErrPsw_0to10_20-30-30.mat
[12] -----------> With_TWPA_10db_270.png
[13] -----------> XAndYErrFunc_19-09-49.mat
[14] -----------> Y_ErrPsw_0to20_11-30-00.mat
[15] -----------> Y_ErrPsw_0to20_13-59-26.mat
[16] -----------> Y_ErrPsw_0to20_14-02-44.mat
[17] -----------> Y_ErrPsw_0to20_14-07-08.mat
[18] -----------> Y_ErrPsw_0to20_14-07-28.mat
[19] -----------> Y_ErrPsw_0to20_14-16-21.mat
[20] -----------> Y_ErrPsw_0to20_15-03-46.mat
[21] -----------> Y_ErrPsw_0to20_16-03-04.mat
[22] -----------> Y_ErrPsw_0to2

In [4]:
mat = scipy.io.loadmat(initial_folder+"\\"+files[0])
# mat

ValueError: Unknown mat file type, version 32, 32

In [5]:
epsilon = mat['epsilon']
slope_epsilon = mat['slope_epsilon']
freq_y = mat['freq_y']
slope_fft = mat['slope_fft']
err = mat['err']
epsilon = mat['epsilon']
att_err = np.arange(0,20)
f_res4 = mat['f_res4'][0]
fft_fr = mat['fft_fr'][0]
XY = mat['XY'][0]

slope = mat['slope'][0]
pha_sweep = mat['pha_sweep'][0]


TypeError: 'function' object is not subscriptable

In [6]:
%matplotlib qt
fig, ax = plt.subplots(2, 4, figsize=(25,13))
plt.rcParams['axes.facecolor'] = 'white'
ax[0,0].set_title('Error function')
ax[0,1].set_title('Slopes of e')
ax[1,0].set_title('Raw noise spectrum')
ax[1,1].set_title('Noise spectrum')
ax[1,2].set_title('Slopes of noise')
ax[0,2].set_title('Phases')
ax[1,3].set_title('b offset in fft fit')
ax[0,3].set_title('Input_pow')
# ax[0,2].set_yscale('log', base=10,)
ax[0,1].set_yscale('log', base=10,)

ax[0,0].set_xlabel('freq offset')
ax[0,1].set_xlabel('Attenuation')
ax[0,2].set_xlabel('Attenuation')
ax[0,3].set_xlabel('Attenuation')

ax[1,2].set_xlabel('Attenuation')
ax[1,3].set_xlabel('Attenuation')

# ax[1,1].set_yscale('log', base=10,)
ax[0,2].grid()
ax[0,1].grid()
ax[0,0].grid()
ax[1,1].grid()
ax[1,0].grid()
ax[1,2].grid()
ax[0,3].grid()
ax[1,3].grid()

start = 2
stop = -1
slopes_a = np.array([])
slopes_ff = np.array([])
b_ff = np.array([])
d = ((freq_y-f_res4)*1000)[0]

cmap = plt.get_cmap('jet_r')
x = np.linspace(0, 1, int(len(att_err)))
colors = cmap(x)

for i in range(len(att_err)):
    ax[0,0].plot(d, epsilon[:,i], color = colors[i], label = '-{} db'.format(att_err[i]))
    
    a, b = linear_fit(d[start:stop], epsilon[start:stop, i])
    slopes_a = np.append(slopes_a, a)
    ax[0,0].plot(d[start:stop], linear_func(d[start:stop], a, b), 'b', ':',  linewidth = 1, alpha = 0.7)
    
    c = slopes_a[i]**2
    c1 = -2*np.log10(abs(slopes_a[i]))
    fft_pow = err[:,i]/10+c1
    
    ax[1,0].plot(np.log10(fft_fr[1:]), err[1:,i],  color = colors[i], label = '-{} db'.format(att_err[i]))
    
    ax[1,1].plot(np.log10(fft_fr[1:]), fft_pow[1:], color = colors[i],  label = '-{} db'.format(att_err[i]))
    
    sta = 2
    sto = 50
    a, b = linear_fit(np.log10(fft_fr[sta:-sto]), fft_pow[sta:-sto])
    slopes_ff = np.append(slopes_ff, a)
    b_ff = np.append(b_ff, b)
    ax[1,1].plot(np.log10(fft_fr[sta:-sto]), linear_func(np.log10(fft_fr[sta:-sto]), a, b), 'b', ':',  linewidth = 1, alpha = 0.7)


ax[0,1].plot(np.arange(0,len(slopes_a)), abs(slopes_a/10))

ax[0,2].plot(np.arange(0,len(slope)), pha_sweep)


ax[1,3].plot(np.arange(0,len(slope)), abs(b_ff/10))
ax[0,3].plot(np.arange(0,len(slope)), XY)

ax[1,2].plot(np.arange(0,len(att_err[:-1])), slopes_ff[:-1])
# a, b = linear_fit(20-att_err[:-1],  slopes_ff[:-1])
# ax[1,2].plot(20-att_err[:-1],linear_func(20-att_err[:-1], a, b), label = 'Fit: a={}'.format(np.round(a,4)))
ax[1,2].legend()

# ax[0,0].set_ylim(-1e-3, 1e-3)

NameError: name 'freq_y' is not defined